Word frequency

In [6]:
reader = open("europarl-v7.hu-en.hu", "r")
writer = open("word_freq.txt", "w")

word_list = {}
word_count_list = {}

for word in reader.read().split():
    if word not in word_list:
        word_list[word] = 1
    else:
        word_list[word] += 1
        
for word in word_list.keys():
    count = word_list[word]
    
    if count not in word_count_list:
        word_count_list[count] = [word]
        
    else:
        word_count_list[count].append(word)
        
        
for count in sorted(word_count_list, reverse = True):
    for word in sorted(word_count_list[count]):
        writer.write("{0}\t{1}\n".format(word,count))
        



reader.close()
writer.close()


Word frequency with Counter

In [7]:
from collections import Counter


def wordfreq(data): 
    word_counter = Counter()

    for word in data:
        word_counter[word] += 1 
    
    return word_counter
    


Character frequency

In [8]:
reader = open("europarl-v7.hu-en.hu", "r")
writer = open("char_freq.txt", "w")

character_list = {}
character_count_list = {}

for word in reader.read().split():
    for char in list(word):
        
        if char not in character_list:
            character_list[char] = 1
        else:
            character_list[char] += 1
            

for char in character_list.keys():
    count = character_list[char]
    
    if count not in character_count_list:
        character_count_list[count] = [char]
    else:
        character_count_list[count].append(char)
    
            
for count in sorted(character_count_list, reverse = True):
    for char in sorted(character_count_list[count]):
        writer.write("{0}\t{1}\n".format(char,count))
        

        
reader.close()
writer.close()

Character frequency with Counter

In [9]:
def char_freq(data):
    character_counter = Counter()

    for word in data:
        if word == "_NUM_":
            character_counter["_NUM_"] +=1
        for char in list(word):
            if char.isdigit():
                character_counter["_NUM_"] +=1
            else:
                character_counter[char] += 1
            

    return character_counter

Exercise 2

In [10]:
import string
def clean_text(inputfile):
    with open(inputfile,"r") as f:
        words = f.read().split()
        for i in range(len(words)):
            words[i] = words[i].lower()
            for c in string.punctuation:
                words[i] = words[i].replace(c,"")
            if words[i].isdigit():
                words[i] = "_NUM_"
    
    return words

Cleaning the corpus.

In [11]:
clean_input = clean_text("europarl-v7.hu-en.hu")

Word count on cleaned corpus.

In [12]:
wc = wordfreq(clean_input)

In [13]:
with open("cleaned_words.txt", "w") as f:
    
    for data in wc.most_common():
            f.write("{0}\t{1}\n".format(data[0],data[1]))

Character count on cleaned corpus.

In [14]:
cc = char_freq(clean_input)

In [15]:
with open("cleaned_chars.txt", "w") as f:
    for data in cc.most_common():
        f.write("{0}\t{1}\n".format(data[0],data[1]))

Exercise 3

In [16]:
with open("europarl-v7.hu-en.hu", "r") as r:
    words = r.read()


In [22]:
def filter_rare_words(input_words, rare_threshold = None):
    if rare_threshold is None:
        rare_threshold = 5
    
    with open("filtered_text.txt","w") as w:
        for line in input_words:
            for word in line.split():
                if wc[word] <= rare_threshold:
                    line = line.replace(word,"_RARE_ ")
            w.write(line) 
        
        

In [24]:
filter_rare_words(words,10)